# AWS Postgres RDS Debug Notebook

This notebook allows to run queries manually against the AWS postgres DB.

In [2]:
import sys
sys.path.insert(0, '/Users/ferdi/Documents/agent-copilot/src')

import psycopg2
import psycopg2.extras
from urllib.parse import urlparse
import pandas as pd
from datetime import datetime
import os

# Import the database module
from aco.server import db

2025-11-26 12:11:08,666 - ACO - INFO - Using PostgreSQL database backend


## Check Database Connection

(You don't really need to run this. I just had to debug something once and wanted to keep it around)

In [7]:
# EMERGENCY NETWORK DIAGNOSIS - Run this first!
print("🚨 Basic network connectivity test...")

import socket
import subprocess
from urllib.parse import urlparse
from aco.common.constants import DATABASE_URL

if DATABASE_URL:
    parsed = urlparse(DATABASE_URL)
    host = parsed.hostname
    port = parsed.port or 5432
    
    print(f"Target: {host}:{port}")
    
    # Test 1: DNS resolution (should be instant)
    print(f"\n1️⃣ Testing DNS resolution for {host}...")
    try:
        import socket
        ip = socket.gethostbyname(host)
        print(f"✅ DNS resolved to: {ip}")
    except Exception as e:
        print(f"❌ DNS resolution failed: {e}")
        print("→ This means the hostname is invalid or DNS is broken")
    
    # Test 2: Ping test (basic connectivity)
    print(f"\n2️⃣ Testing ping to {host}...")
    try:
        result = subprocess.run(['ping', '-c', '2', host], 
                              capture_output=True, text=True, timeout=10)
        if result.returncode == 0:
            print("✅ Ping successful")
            # Extract timing info
            lines = result.stdout.split('\n')
            for line in lines:
                if 'time=' in line:
                    print(f"   {line.strip()}")
        else:
            print("❌ Ping failed")
            print(f"   Error: {result.stderr}")
    except subprocess.TimeoutExpired:
        print("❌ Ping timeout - host unreachable")
    except Exception as e:
        print(f"❌ Ping error: {e}")
    
    # Test 3: Telnet-style connection test (fastest way to test port)
    print(f"\n3️⃣ Testing port connectivity to {host}:{port}...")
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(5)  # Very short timeout
        result = sock.connect_ex((host, port))
        sock.close()
        
        if result == 0:
            print("✅ Port is reachable")
        else:
            print(f"❌ Port connection failed (error {result})")
            print("   → Database server may be down or port blocked")
    except Exception as e:
        print(f"❌ Socket error: {e}")
        
else:
    print("❌ No DATABASE_URL found")

# Test 4: General internet connectivity 
print(f"\n4️⃣ Testing general internet connectivity...")
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(3)
    result = sock.connect_ex(("8.8.8.8", 53))  # Google DNS
    sock.close()
    
    if result == 0:
        print("✅ Internet connectivity OK")
    else:
        print("❌ No internet connectivity")
        print("   → Check your network connection")
except Exception as e:
    print(f"❌ Internet test failed: {e}")

print(f"\n🔍 If all tests pass but db.get_conn() hangs:")
print("   → The PostgreSQL server is likely overloaded or in maintenance")
print("   → Check AWS RDS console for instance status")
print("   → Try connecting from a different network/location")

🚨 Basic network connectivity test...
Target: workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com:5432

1️⃣ Testing DNS resolution for workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com...
✅ DNS resolved to: 98.90.57.89

2️⃣ Testing ping to workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com...
❌ Ping timeout - host unreachable

3️⃣ Testing port connectivity to workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com:5432...
✅ Port is reachable

4️⃣ Testing general internet connectivity...
✅ Internet connectivity OK

🔍 If all tests pass but db.get_conn() hangs:
   → The PostgreSQL server is likely overloaded or in maintenance
   → Check AWS RDS console for instance status
   → Try connecting from a different network/location


## Database Schema

In [11]:
# Show complete database schema information
print("📊 DATABASE SCHEMA OVERVIEW\n" + "="*50)

# Get all tables in the database
tables_query = """
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public' 
    ORDER BY table_name
"""
tables = db.query_all(tables_query)

if not tables:
    print("❌ No tables found in the database")
else:
    print(f"✅ Found {len(tables)} tables in the database:\n")
    
    for table_info in tables:
        table_name = table_info['table_name']
        print(f"\n📋 Table: {table_name.upper()}")
        print("-" * 40)
        
        # Get columns for each table
        columns_query = """
            SELECT 
                column_name,
                data_type,
                character_maximum_length,
                is_nullable,
                column_default
            FROM information_schema.columns 
            WHERE table_name = %s AND table_schema = 'public'
            ORDER BY ordinal_position
        """
        columns = db.query_all(columns_query, (table_name,))
        
        # Display columns
        for col in columns:
            col_name = col['column_name']
            col_type = col['data_type']
            
            # Add length info if applicable
            if col['character_maximum_length']:
                col_type += f"({col['character_maximum_length']})"
            
            # Add NULL/NOT NULL
            nullable = "NULL" if col['is_nullable'] == 'YES' else "NOT NULL"
            
            # Add default value if exists
            default = f" DEFAULT {col['column_default']}" if col['column_default'] else ""
            
            print(f"  • {col_name}: {col_type} {nullable}{default}")
        
        # Get primary key constraints
        pk_query = """
            SELECT column_name
            FROM information_schema.key_column_usage kcu
            JOIN information_schema.table_constraints tc 
                ON kcu.constraint_name = tc.constraint_name
            WHERE tc.table_name = %s 
                AND tc.constraint_type = 'PRIMARY KEY'
                AND tc.table_schema = 'public'
            ORDER BY kcu.ordinal_position
        """
        pk_cols = db.query_all(pk_query, (table_name,))
        
        if pk_cols:
            pk_names = [col['column_name'] for col in pk_cols]
            print(f"\n  🔑 Primary Key: ({', '.join(pk_names)})")
        
        # Get foreign key constraints
        fk_query = """
            SELECT
                kcu.column_name,
                ccu.table_name AS foreign_table_name,
                ccu.column_name AS foreign_column_name
            FROM information_schema.table_constraints AS tc
            JOIN information_schema.key_column_usage AS kcu
                ON tc.constraint_name = kcu.constraint_name
            JOIN information_schema.constraint_column_usage AS ccu
                ON ccu.constraint_name = tc.constraint_name
            WHERE tc.constraint_type = 'FOREIGN KEY' 
                AND tc.table_name = %s
                AND tc.table_schema = 'public'
        """
        fk_cols = db.query_all(fk_query, (table_name,))
        
        if fk_cols:
            print("\n  🔗 Foreign Keys:")
            for fk in fk_cols:
                print(f"    • {fk['column_name']} → {fk['foreign_table_name']}.{fk['foreign_column_name']}")
        
        # Get unique constraints
        unique_query = """
            SELECT column_name
            FROM information_schema.key_column_usage kcu
            JOIN information_schema.table_constraints tc 
                ON kcu.constraint_name = tc.constraint_name
            WHERE tc.table_name = %s 
                AND tc.constraint_type = 'UNIQUE'
                AND tc.table_schema = 'public'
        """
        unique_cols = db.query_all(unique_query, (table_name,))
        
        if unique_cols:
            unique_names = [col['column_name'] for col in unique_cols]
            print(f"\n  ⭐ Unique Constraints: {', '.join(unique_names)}")
        
        # Get indexes
        index_query = """
            SELECT 
                indexname,
                indexdef
            FROM pg_indexes
            WHERE tablename = %s
                AND schemaname = 'public'
        """
        indexes = db.query_all(index_query, (table_name,))
        
        if indexes:
            print("\n  📍 Indexes:")
            for idx in indexes:
                # Skip primary key index (already shown above)
                if not idx['indexname'].endswith('_pkey'):
                    print(f"    • {idx['indexname']}")
        
        # Get row count
        count_query = f"SELECT COUNT(*) as count FROM {table_name}"
        count_result = db.query_all(count_query)
        row_count = count_result[0]['count'] if count_result else 0
        print(f"\n  📈 Row Count: {row_count:,}")

print("\n" + "="*50)
print("✅ Schema overview complete")

📊 DATABASE SCHEMA OVERVIEW
✅ Found 3 tables in the database:


📋 Table: ATTACHMENTS
----------------------------------------
  • file_id: text NOT NULL
  • session_id: text NULL
  • line_no: integer NULL
  • content_hash: text NULL
  • file_path: text NULL
  • taint: text NULL

  🔑 Primary Key: (file_id)

  🔗 Foreign Keys:
    • session_id → experiments.session_id

  📍 Indexes:
    • attachments_content_hash_idx

  📈 Row Count: 0

📋 Table: EXPERIMENTS
----------------------------------------
  • session_id: text NOT NULL
  • parent_session_id: text NULL
  • graph_topology: text NULL
  • color_preview: text NULL
  • timestamp: timestamp without time zone NULL DEFAULT CURRENT_TIMESTAMP
  • cwd: text NULL
  • command: text NULL
  • environment: text NULL
  • code_hash: text NULL
  • name: text NULL
  • success: text NULL
  • notes: text NULL
  • log: text NULL

  🔑 Primary Key: (session_id)

  🔗 Foreign Keys:
    • parent_session_id → experiments.session_id

  ⭐ Unique Constraints: parent

## List Table Entries

### experiments table

In [10]:
# Get all experiments
experiments = db.query_all(
    "SELECT session_id, parent_session_id, name, timestamp, success, notes FROM experiments ORDER BY timestamp DESC LIMIT 20"
)

if experiments:
    df_experiments = pd.DataFrame(experiments)
    print(f"Found {len(experiments)} experiments:")
    display(df_experiments)
else:
    print("No experiments found in database")

2025-11-17 02:34:36,496 - ACO - WARNING - Database connection error on attempt 1: SSL connection has been closed unexpectedly

2025-11-17 02:34:37,109 - ACO - DEBUG - Database schema initialized
2025-11-17 02:34:37,111 - ACO - INFO - Initialized PostgreSQL connection to workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com


No experiments found in database


### llm_calls table

In [12]:
# Get recent LLM calls
llm_calls = db.query_all(
    "SELECT session_id, node_id, api_type, timestamp FROM llm_calls ORDER BY timestamp DESC LIMIT 20"
)

if llm_calls:
    df_llm = pd.DataFrame(llm_calls)
    print(f"Found {len(llm_calls)} recent LLM calls:")
    display(df_llm)
else:
    print("No LLM calls found")

No LLM calls found


In [10]:
print("🔧 RESETTING DATABASE SCHEMA TO MATCH postgres.py\n" + "="*50)

try:
    # Use the existing db connection from the first cell
    conn = db.get_conn()
    
    # First, commit or rollback any existing transaction
    try:
        conn.rollback()
        print("1️⃣ Rolled back any existing transaction")
    except:
        pass
    
    # Now we can set autocommit
    conn.autocommit = True
    c = conn.cursor()
    print("2️⃣ Using existing database connection (autocommit mode)")
    
    # Drop all existing tables in correct order (respecting foreign keys)
    print("\n3️⃣ Dropping existing tables...")
    c.execute("DROP TABLE IF EXISTS attachments CASCADE")
    print("   ✅ Dropped attachments table")
    c.execute("DROP TABLE IF EXISTS llm_calls CASCADE")
    print("   ✅ Dropped llm_calls table")
    c.execute("DROP TABLE IF EXISTS experiments CASCADE")
    print("   ✅ Dropped experiments table")
    c.execute("DROP TABLE IF EXISTS users CASCADE")  # Also drop users if it exists
    print("   ✅ Dropped users table (if existed)")
    
    # Create experiments table (exactly as in postgres.py)
    print("\n4️⃣ Creating tables with schema from postgres.py...")
    c.execute(
        """
        CREATE TABLE IF NOT EXISTS experiments (
            session_id TEXT PRIMARY KEY,
            parent_session_id TEXT,
            graph_topology TEXT,
            color_preview TEXT,
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            cwd TEXT,
            command TEXT,
            environment TEXT,
            code_hash TEXT,
            name TEXT,
            success TEXT CHECK (success IN ('', 'Satisfactory', 'Failed')),
            notes TEXT,
            log TEXT,
            FOREIGN KEY (parent_session_id) REFERENCES experiments (session_id),
            UNIQUE (parent_session_id, name)
        )
        """
    )
    print("   ✅ Created experiments table")
    
    # Create llm_calls table (NOTE: output is TEXT, not BYTEA!)
    c.execute(
        """
        CREATE TABLE IF NOT EXISTS llm_calls (
            session_id TEXT,
            node_id TEXT,
            input BYTEA,
            input_hash TEXT,
            input_overwrite BYTEA,
            output TEXT,
            color TEXT,
            label TEXT,
            api_type TEXT,
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            PRIMARY KEY (session_id, node_id),
            FOREIGN KEY (session_id) REFERENCES experiments (session_id)
        )
        """
    )
    print("   ✅ Created llm_calls table (with output as TEXT)")
    
    # Create attachments table
    c.execute(
        """
        CREATE TABLE IF NOT EXISTS attachments (
            file_id TEXT PRIMARY KEY,
            session_id TEXT,
            line_no INTEGER,
            content_hash TEXT,
            file_path TEXT,
            taint TEXT,
            FOREIGN KEY (session_id) REFERENCES experiments (session_id)
        )
        """
    )
    print("   ✅ Created attachments table")
    
    # Create indexes (exactly as in postgres.py)
    print("\n5️⃣ Creating indexes...")
    c.execute(
        """
        CREATE INDEX IF NOT EXISTS attachments_content_hash_idx ON attachments(content_hash)
        """
    )
    print("   ✅ Created attachments_content_hash_idx")
    
    c.execute(
        """
        CREATE INDEX IF NOT EXISTS original_input_lookup ON llm_calls(session_id, input_hash)
        """
    )
    print("   ✅ Created original_input_lookup index")
    
    c.execute(
        """
        CREATE INDEX IF NOT EXISTS experiments_timestamp_idx ON experiments(timestamp DESC)
        """
    )
    print("   ✅ Created experiments_timestamp_idx")
    
    # Verify the schema
    print("\n6️⃣ Verifying schema...")
    c.execute("""
        SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = 'llm_calls' AND column_name = 'output'
    """)
    output_col = c.fetchone()
    
    if output_col:
        output_type = output_col[1]
        if output_type == 'text':
            print(f"   ✅ Output column is TEXT (matching postgres.py)")
        else:
            print(f"   ⚠️ Output column is {output_type} (expected TEXT)")
    
    # Get table count
    c.execute("""
        SELECT COUNT(*) FROM information_schema.tables 
        WHERE table_schema = 'public'
    """)
    table_count = c.fetchone()[0]
    
    print(f"\n✅ SCHEMA RESET COMPLETE!")
    print(f"   • {table_count} tables created")
    print(f"   • All indexes created")
    print(f"   • Schema now matches postgres.py exactly")
    print(f"\n⚠️ IMPORTANT: The 'output' column in llm_calls is TEXT (not BYTEA)")
    print(f"   This matches the original postgres.py schema.")
    
    # Reset autocommit back to False as per postgres.py default
    conn.autocommit = False
    print(f"   • Autocommit reset to False")
    
except Exception as e:
    print(f"\n❌ Schema reset failed: {e}")
    print(f"   Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()
    
    # Try to restore connection state
    try:
        conn.rollback()
        conn.autocommit = False
    except:
        pass

🔧 RESETTING DATABASE SCHEMA TO MATCH postgres.py
1️⃣ Rolled back any existing transaction
2️⃣ Using existing database connection (autocommit mode)

3️⃣ Dropping existing tables...
   ✅ Dropped attachments table
   ✅ Dropped llm_calls table
   ✅ Dropped experiments table
   ✅ Dropped users table (if existed)

4️⃣ Creating tables with schema from postgres.py...
   ✅ Created experiments table
   ✅ Created llm_calls table (with output as TEXT)
   ✅ Created attachments table

5️⃣ Creating indexes...
   ✅ Created attachments_content_hash_idx
   ✅ Created original_input_lookup index
   ✅ Created experiments_timestamp_idx

6️⃣ Verifying schema...
   ✅ Output column is TEXT (matching postgres.py)

✅ SCHEMA RESET COMPLETE!
   • 3 tables created
   • All indexes created
   • Schema now matches postgres.py exactly

⚠️ IMPORTANT: The 'output' column in llm_calls is TEXT (not BYTEA)
   This matches the original postgres.py schema.
   • Autocommit reset to False


In [5]:
# Verify the schema change was successful
final_schema = db.query_all(
    """
    SELECT column_name, data_type, is_nullable
    FROM information_schema.columns 
    WHERE table_name = 'llm_calls' 
    ORDER BY ordinal_position
    """
)

print("Final llm_calls table schema:")
for col in final_schema:
    status = ""
    if col['column_name'] == 'output':
        if col['data_type'] == 'bytea':
            status = " ✅"
        else:
            status = " ❌"
    print(f"  {col['column_name']}: {col['data_type']} ({'NULL' if col['is_nullable'] == 'YES' else 'NOT NULL'}){status}")

# Test that we can insert and retrieve binary data
print("\nTesting binary data storage...")
try:
    import dill
    test_data = {"test": "data", "number": 42}
    test_pickle = dill.dumps(test_data)
    
    # This should work now without error
    print(f"✅ Successfully created pickle data: {len(test_pickle)} bytes")
    print(f"✅ Can load pickle back: {dill.loads(test_pickle)}")
    print("Migration appears successful!")
    
except Exception as e:
    print(f"❌ Error with pickle test: {e}")

2025-11-17 02:34:14,429 - ACO - DEBUG - Database schema initialized
2025-11-17 02:34:14,430 - ACO - INFO - Initialized PostgreSQL connection to workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com


Final llm_calls table schema:
  session_id: text (NOT NULL)
  node_id: text (NOT NULL)
  input: bytea (NULL)
  input_hash: text (NULL)
  input_overwrite: bytea (NULL)
  output: bytea (NULL) ✅
  color: text (NULL)
  label: text (NULL)
  api_type: text (NULL)
  timestamp: timestamp without time zone (NULL)

Testing binary data storage...
✅ Successfully created pickle data: 41 bytes
✅ Can load pickle back: {'test': 'data', 'number': 42}
Migration appears successful!
